In [9]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
from scipy import stats
import time

In [11]:
posts = pd.read_pickle('./data/all_posts.pickle')
print(len(posts))
posts.head()

1046958


,created_utc,id,author,title,selftext,subreddit,link,score,upvote_ratio,author_flair_text,num_comments,removed_by_category,url,divisiveness
0,1588242127,gat3o9,TransGirl757,"If we get approved by 100k people in 1 month, ...",NaN,progressive,/r/progressive/comments/gat3o9/if_we_get_appro...,1,1.0,NaN,0,moderator,https://petitions.whitehouse.gov/petition/give...,1
1,1588222576,gapbez,shallah,Elizabeth Warren and Alexandria Ocasio-Cortez ...,NaN,progressive,/r/progressive/comments/gapbez/elizabeth_warre...,1,1.0,NaN,0,moderator,https://theweek.com/speedreads/911377/elizabet...,1
2,1588216466,ganw7m,shallah,"Gov Whitmer Announces ""Futures for Frontliners...",NaN,progressive,/r/progressive/comments/ganw7m/gov_whitmer_ann...,1,1.0,NaN,5,NaN,"https://www.michigan.gov/whitmer/0,9309,7-387-...",1
3,1588200464,gajmqr,thinkB4WeSpeak,Could pandemic rekindle labor movement?,NaN,progressive,/r/progressive/comments/gajmqr/could_pandemic_...,1,1.0,NaN,2,NaN,https://finance-commerce.com/2020/04/could-pan...,1
4,1588197018,gaikq1,davyjoneses,Found on r/therightcantmeme. Should the govern...,NaN,progressive,/r/progressive/comments/gaikq1/found_on_rtheri...,1,1.0,NaN,1,moderator,https://np.reddit.com/r/TheRightCantMeme/comme...,1


In [4]:
file = "20200713_20200813"
fname = './data/comments/{}_all_comments_roberta.pickle'.format(file)
comments = pd.read_pickle(fname)
print(len(comments))
comments.head()

2834240


,created_utc,parent_id,link_id,id,author,body,subreddit,link,score,author_flair_text,...,anger,you,we,swear,civility,personal_attack,third_party_attack,aspersion,vulgarity,stereotype
2,1597373600,t3_i9azph,t3_i9azph,g1eh32p,Gorehog,Damn. Now the dems got my vote.,uspolitics,/r/uspolitics/comments/i9azph/birtherism_again...,1,NaN,...,0,0,0,0,0.003164,0.000097,0.000886,0.000473,0.000303,0.000359
3,1597371202,t3_i9cyd3,t3_i9cyd3,g1ecno4,id10t_you,He’s playing the hits. The dotard devotees wil...,uspolitics,/r/uspolitics/comments/i9cyd3/trump_falsely_cl...,2,NaN,...,0,0,0,0,0.009717,0.000095,0.001041,0.000518,0.000413,0.000553
4,1597370953,t3_i9d0i8,t3_i9d0i8,g1ec7jm,id10t_you,"We want voting rights for *everyone*, regardle...",uspolitics,/r/uspolitics/comments/i9d0i8/trump_economic_a...,1,NaN,...,0,0,0,0,0.049601,0.000107,0.000960,0.001068,0.000409,0.000567
5,1597370775,t3_i9azph,t3_i9azph,g1ebw96,harrynutzach,"Actually, Newsweek started this.... not the Tr...",uspolitics,/r/uspolitics/comments/i9azph/birtherism_again...,-3,NaN,...,1,0,0,0,0.004385,0.000109,0.001074,0.001139,0.000395,0.000214
6,1597369616,t1_g1e9ofi,t3_i9cqjt,g1e9szw,Projectrage,https://youtu.be/o-0lAhnoDlU,uspolitics,/r/uspolitics/comments/i9cqjt/senate_leaves_un...,1,NaN,...,0,0,0,0,0.007214,0.000212,0.001147,0.000595,0.001417,0.000277


In [5]:
comments.columns

Index(['created_utc', 'parent_id', 'link_id', 'id', 'author', 'body',
       'subreddit', 'link', 'score', 'author_flair_text', 'comment_len',
       'compound_sentiment', 'categories', 'i', 'posemo', 'social', 'negemo',
       'sad', 'negate', 'anger', 'you', 'we', 'swear', 'civility',
       'personal_attack', 'third_party_attack', 'aspersion', 'vulgarity',
       'stereotype'],
      dtype='object')

In [8]:
survival_data_fname = './data/survival_data.pickle'
if os.path.isfile(survival_data_fname):
    survival_data = pd.read_pickle(survival_data_fname)    
    survival_data = survival_data.reset_index()
    survival_data = set(survival_data.sort_values('author').author)
    survival_data = pd.DataFrame(survival_data,columns=['author']).sort_values('author').reset_index()
survival_data

,index,author
0,85248,-----iMartijn-----
1,9990,---00G---
2,73289,---77---
3,100552,---Tsing__Tao---
4,21594,---deadman---
...,...,...
119047,45305,zzziltoid
119048,24397,zzzpranger
119049,21036,zzzwrite
119050,55804,zzzz1234zzzz


In [13]:
feb_2020_timestamp = 1580533200
feb_2021_timestamp = 1612155600

count = 0

# days = [29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 30]
timestamps = [datetime.utcfromtimestamp(feb_2020_timestamp)]
for i in range(1, 53):
  timestamps.append( timestamps[i-1] + timedelta(days=7))

timestamps = [i.timestamp() for i in timestamps]
print(timestamps)

def compute_binned_statistics(exp, fields, row, prefix="sent",use_liwc=False, liwc=None):
  global count
  values = []
  for f in fields:
    values.append(exp[f])
  if use_liwc:
    fields.extend(liwc.columns)
    for i in liwc.columns:
      values.append(liwc[i])
  res = stats.binned_statistic(exp['created_utc'], values, 'mean', bins=timestamps)[0]
  for index, r in enumerate(res):
    row[prefix+"_"+fields[index]+"_hist"] = np.nan_to_num(r)
  
categories = ['vulgarity', 'third_party_attack', 'stereotype', 'personal_attack', 'civility', 'aspersion']

def find_comment_activities(posts_df, comments_df, row):
  global count, program_starts
  if count % 100 == 0:
    now = time.time()
    print("It has been {} seconds since the loop started to get to {}".format(now - program_starts, count))
    program_starts = time.time()
  count += 1
  # exposure
  temp_posts = posts_df[posts_df.author == row.author]
  row['post_week_hist'] = np.histogram(temp_posts['created_utc'], bins=timestamps)[0]
  # if len(temp_posts) > 0:
  #   res = stats.binned_statistic(temp_posts['created_utc'], [temp_posts['is_left'], temp_posts['is_right']], 'sum', bins=timestamps)[0]
  #   row['left_post_hist'] = res[0]
  #   row['right_post_hist'] = res[1]

  temp_df = comments_df[comments_df.link_id.str[3:].isin(temp_posts.id)]
  row['post_comment_num_hist'] = np.histogram(temp_df['created_utc'], bins=timestamps)[0]
  if len(temp_df) > 0:
    compute_binned_statistics(temp_df,  categories, row, 'post')

  # comments
  temp_df = comments_df[comments_df.author == row.author]
  row['comment_week_hist'] = np.histogram(temp_df['created_utc'], bins=timestamps)[0]
  if len(temp_df) > 0:
  #   liwc = temp_df.categories.apply(pd.Series).fillna(0).astype(int)
    compute_binned_statistics(temp_df,  categories, row, 'sent')

    # immediate response
    response_df = comments_df[comments_df.parent_id.str[3:].isin(temp_df.id)]
    if len(response_df) > 0:
      row['comment_response_num_hist'] = np.histogram(temp_df['created_utc'], bins=timestamps)[0]
      compute_binned_statistics(response_df, categories, row, 'comment')
  return row

[1580533200.0, 1581138000.0, 1581742800.0, 1582347600.0, 1582952400.0, 1583557200.0, 1584162000.0, 1584766800.0, 1585371600.0, 1585976400.0, 1586581200.0, 1587186000.0, 1587790800.0, 1588395600.0, 1589000400.0, 1589605200.0, 1590210000.0, 1590814800.0, 1591419600.0, 1592024400.0, 1592629200.0, 1593234000.0, 1593838800.0, 1594443600.0, 1595048400.0, 1595653200.0, 1596258000.0, 1596862800.0, 1597467600.0, 1598072400.0, 1598677200.0, 1599282000.0, 1599886800.0, 1600491600.0, 1601096400.0, 1601701200.0, 1602306000.0, 1602910800.0, 1603515600.0, 1604120400.0, 1604725200.0, 1605330000.0, 1605934800.0, 1606539600.0, 1607144400.0, 1607749200.0, 1608354000.0, 1608958800.0, 1609563600.0, 1610168400.0, 1610773200.0, 1611378000.0, 1611982800.0]


In [14]:
survival_data_dup = survival_data.drop('index', axis=1)
survival_data_dup = survival_data_dup.reset_index()

file_dir = './data/survival_temp/'+file
print(file_dir)
if not os.path.exists(file_dir):
    os.makedirs(file_dir)
i = 0
step_size = 10
program_starts = time.time()
count = 0
for i in range(count, len(survival_data_dup), step_size):
  survival_data_cur = survival_data_dup.loc[i:i+step_size-1]
  posts_cur = posts[posts.author.isin(survival_data_cur.author)]
  comments_cur = comments[(comments.author.isin(survival_data_cur.author))]
  comments_cur = comments[(comments.author.isin(survival_data_cur.author) | comments.link_id.isin(posts_cur.id) | comments.link_id.isin(comments_cur.link_id))]
  survival_data_cur = survival_data_cur.apply(lambda row: find_comment_activities(posts_cur, comments_cur, row), axis=1)
  survival_data_cur.to_pickle(file_dir +'/survival_data_{}.pickle'.format(i))

./data/survival_temp/20200713_20200813
It has been 1.1824066638946533 seconds since the loop started to get to 0
It has been 8.954460144042969 seconds since the loop started to get to 100
It has been 10.188411474227905 seconds since the loop started to get to 200
It has been 9.019246339797974 seconds since the loop started to get to 300
It has been 10.312079191207886 seconds since the loop started to get to 400
It has been 8.706517219543457 seconds since the loop started to get to 500
It has been 10.519818305969238 seconds since the loop started to get to 600
It has been 9.648820877075195 seconds since the loop started to get to 700
It has been 8.696343421936035 seconds since the loop started to get to 800
It has been 8.913622856140137 seconds since the loop started to get to 900
It has been 10.71904468536377 seconds since the loop started to get to 1000
It has been 10.204350471496582 seconds since the loop started to get to 1100
It has been 9.870379209518433 seconds since the loop sta